<a href="https://colab.research.google.com/github/roni762583/bible-data-science.github.io/blob/master/CleanBible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math


LOAD DATA - Ref: https://medium.com/@yuraist/how-to-upload-your-own-dataset-into-google-colab-e228727c87e9

In [2]:
#clone data
! git clone https://github.com/roni762583/bible-data-science.github.io.git

Cloning into 'bible-data-science.github.io'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 178 (delta 11), reused 0 (delta 0), pack-reused 152
Receiving objects: 100% (178/178), 7.60 MiB | 8.23 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [3]:
# copy my datafile
! cp /content/bible-data-science.github.io/data/t3utf.dat datafile

In [4]:
! ls -la
# delete unused parts
! rm -rf /content/bible-data-science.github.io/
! echo '--------------------------------------'
! ls -la

total 5652
drwxr-xr-x 1 root root    4096 Jan 19 17:13 .
drwxr-xr-x 1 root root    4096 Jan 19 17:11 ..
drwxr-xr-x 7 root root    4096 Jan 19 17:13 bible-data-science.github.io
drwxr-xr-x 1 root root    4096 Jan 15 17:28 .config
-rw-r--r-- 1 root root 5763655 Jan 19 17:13 datafile
drwxr-xr-x 1 root root    4096 Jan 15 17:28 sample_data
--------------------------------------
total 5648
drwxr-xr-x 1 root root    4096 Jan 19 17:13 .
drwxr-xr-x 1 root root    4096 Jan 19 17:11 ..
drwxr-xr-x 1 root root    4096 Jan 15 17:28 .config
-rw-r--r-- 1 root root 5763655 Jan 19 17:13 datafile
drwxr-xr-x 1 root root    4096 Jan 15 17:28 sample_data


In [19]:
# Build Tanach string without non-Hebrew letters characters
# Opening file 
file1 = open('datafile', 'r') 
count = 0
s = '' # basic Hebrew chars string
s2 = '' # non-Hebrew chars string
# dataframe for reshaped data
# df = pd.dataframe()
# loop over file line by line, one verse per line 
for line in file1: 
    count += 1
    #print("Line{}: {}".format(count, line.strip())) 
    #print(line[0]) # first character in line
    #print(line.rfind('|')) # last index of pipe '|' character
    line = (line[line.rfind('|')+1:]).replace('׃','') #remove verse and colon
    line = line.strip() # remove newline char
    line = line.replace(" ", "") # remove spaces
    for c in line:
      if c in {'א', 'ב', 'ג' ,'ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם','נ','ן','ס','ע','פ','ף','צ','ץ','ק','ר','ש','ת'}:
        s += c # concat Hebrew letter to s
      else:
        s2 += c
# print (list(set(s2))) # get unique chars list
print (s[-1]) # print last letter ל
# Closing files 
file1.close()
print(len(s))
print("Last letter of Torah is off (expected index 304805+1), actually at 304805+357: " + str(s[304805+300:304805+357]))
# '\u200d' is Zero-Width Joiner


ל
1202888
Last letter of Torah is off (expected index 304805+1), actually at 304805+357: יוולכלארצוולכלהידהחזקהולכלהמוראהגדולאשרעשהמשהלעיניכלישראל


In [ ]:
# write no spaces string to file - only run once
text_file = open("NoSpacesTanach.dat", "w")
text_file.write(s)
text_file.close()

# delete intermediary datafile
! rm datafile

#see
! ls -la

total 2368
drwxr-xr-x 1 root root    4096 Jan 17 05:33 .
drwxr-xr-x 1 root root    4096 Jan 17 02:30 ..
drwxr-xr-x 1 root root    4096 Jan  8 17:11 .config
-rw-r--r-- 1 root root 2405776 Jan 17 05:33 NoSpacesTanach.dat


In [ ]:
# download result file - only run once
from google.colab import files
files.download('NoSpacesTanach.dat')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# get user input for search term
term1 = input ("Enter Primary Search Term:")

Enter Primary Search Term:עדנה


In [3]:
print(term1)

עדנה


In [11]:
# find indeces of first term letter in bible text
# from: https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string
def find_nth_overlapping(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+1)
        n -= 1
    return start

In [27]:
nthOccurance = 6
index = find_nth_overlapping(s, term1, nthOccurance)
print(s[index-10:index+10])

דהשריאלפיםעדנההשרועמ


In [17]:
print(s[21070:21089])

תיהיתהליעדנהואדניזק


In [29]:
# try to get some ELS 
# from: https://stackoverflow.com/questions/46503865/getting-every-nth-character-in-the-string-in-python
def GetNthLetters(text, n):
    builtstring = ""
    for i in range(0, len(text)):
        if (i + 1) % n == 0:
            # print(text[i])
            builtstring = builtstring + text[i]
    return builtstring


#text = '1234567890123456789012345678901234567890'

nthLetters = GetNthLetters(s, 8) # 1 or larger. is ELS+1 i.e. no skip is ELS of zero, but here it is one.
print(nthLetters[0:100])

רתתרוךוהעםהרוםילןןראחלביאיתיביהיירייליאיהינלההקרוקלויםייםץזץרשעהאעזורווייהשמהקלןןימיםוהיועתתתדהמליון


In [ ]:
# Get nth char in string from index j
def GetELSat(text, n, j):
    text = text[j:]
    print("castrated text: " + text[:50])
    elsstring = ""
    for i in range(0, len(text)):
        if (i + 1) % n == 0:
            # print(text[i])
            elsstring = elsstring + text[i]
    return elsstring